In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the KML file
tree = ET.parse("../data/client-data/jesse-kml.xml")
root = tree.getroot()

records = []
for placemark in root.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text 
    description = placemark.find('{http://www.opengis.net/kml/2.2}description').text
    coordinates = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text 


    coordinates = coordinates.strip().split(",") 
    # longitude,latitude, _ = coordinates # type:ignore https://dwtkns.com/pointplotter/
    records.append({
        "name": name,
        "long_name": description.split("<br>")[0][6:], 
        # some of these shapes are bounding boxes. this would mean the eentity is a county/district, not a school
        "type": "SCHOOL" if len(coordinates) == 3 else "COUNTY", 
        "longitude": float(coordinates[0]),
        "latitude": float(coordinates[1])
    })


In [ ]:
import geopandas as gpd
from shapely.geometry import Point


geometry = [Point(r["longitude"], r["latitude"]) for r in records]
gdf_reachwell = gpd.GeoDataFrame(records, geometry=geometry, crs='EPSG:4326')
# gdf_reachwell = gpd.read_file("reachwell_client_list.json")


In [ ]:

gdf_school_districts = gpd.read_file("../geojson-school-districts.json")

gdf_combine = gpd.sjoin(
    gdf_school_districts, 
    gdf_reachwell[gdf_reachwell["type"] == "SCHOOL"],
    how="inner"
)
# gdf_combine["id_right"].isna().value_counts()
# gdf_combine[["NAME",'long_name']].head(20)


In [ ]:
# after writing file, lok up and add in the county codes manually
gdf_reachwell[gdf_reachwell['type'] == "COUNTY"] 


In [ ]:
pd.DataFrame(
    data={"geo_id": ["9700000US{nces_id}".format(nces_id=nces_id) for nces_id in gdf_combine["id_left"].unique()]}
).to_csv("reachwell-clients.csv",index=False)